# 1 Open and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
from calculate_angle import calculate_angle
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 2. Determining Joints and check angle with timer

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
cap = cv2.VideoCapture('3s.mp4')
timer = 0
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

         # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            print(angle)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            # This is infinite loop inside infinite loop
            # t=8
            # while angle <120:
            #    while t>0:
            #     t = t-1
            #     print(t)

            # Try using FPS cap.get(cv2.CAP_PROP_FPS) this may fix
            if angle <120:
                timer = timer + 1
                if timer == 9:
                    break
            else:
                timer = 0
        except:
            pass
        #Render Countdown
        cv2.rectangle(image, (0,0), (250,100), (245,117,16), -1)
        cv2.putText(image, str(timer),
                    (90,90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()